сделаем RAG pipeline на основе конспектов по курсу машинки

вот сурс:
https://www.overleaf.com/read/gqpbbyphsjwt#4773bd

об обработке данных:
код находится в `RAG/data_parser`

1) Так как у меня конспект, то каждой главе текста соответствует еще и название. Поэтому каждому куску текста кроме своего эмбеддинга сопоставлялся эмбеддинг названия главы. 
2) Эксперименты показали, что на моих данных лучше всего удалить всякие латеховские конструкции и большие формулы. А также поочистить криво поставленные знаки препинания. Это делается в соответствующей функции.
3) По некоторым билетам отсутствуют текстовые ответы, поэтому модель не умеет отвечать на эти вопросы.

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

import torch

from RAG.vectorizer import DBVectirizer
from RAG.data_parser import parse_data
from RAG.index_creator import load_index

os.environ["CUDA_VISIBLE_DEVICES"]="1,2"

torch.cuda.is_available()

True

Парсим данные из директории с LaTeX кодом и сохраняем их в виде csv файла

In [2]:
parse_data()

Загружаем retriever. Код его составления вынесен в отдельный файл. в процессе работы не сохраняю получившуюся модель, поскольку данных немного.

`model_name` указывает, какая модель используется для векторизации текста.


In [3]:
# model_name = "t-tech/T-lite-it-1.0"
model_name = "ai-forever/sbert_large_nlu_ru"

retriever = load_index(model_name = model_name)

Загрузка модели. Используем VLLM  фреймворк, чтобы все быстро работало

In [3]:
from langchain_community.llms import VLLM
# model_name = "ai-forever/sbert_large_nlu_ru"
# model_name = "ai-forever/rugpt3large_based_on_gpt2"
# model_name = "yahma/llama-7b-hf"
model_name = "t-tech/T-lite-it-1.0"

llm_model = VLLM(
    model=model_name,
    device_map="cuda:1",
    max_new_tokens=500,
    enforce_eager=True,
    dtype="bfloat16",
    gpu_memory_utilization=0.8,
    GPU_max_memory= {0: '0GiB', 1: '20GiB'}
)

INFO 01-19 10:44:36 config.py:510] This model supports multiple tasks: {'embed', 'reward', 'generate', 'classify', 'score'}. Defaulting to 'generate'.
INFO 01-19 10:44:36 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='t-tech/T-lite-it-1.0', speculative_config=None, tokenizer='t-tech/T-lite-it-1.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=t-tech/T-l

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 01-19 10:44:42 model_runner.py:1099] Loading model weights took 14.2426 GB
INFO 01-19 10:44:44 worker.py:241] Memory profiling takes 2.09 seconds
INFO 01-19 10:44:44 worker.py:241] the current vLLM instance can use total_gpu_memory (79.25GiB) x gpu_memory_utilization (0.90) = 71.33GiB
INFO 01-19 10:44:44 worker.py:241] model weights take 14.24GiB; non_torch_memory takes 0.14GiB; PyTorch activation peak memory takes 4.35GiB; the rest of the memory reserved for KV Cache is 52.59GiB.
INFO 01-19 10:44:44 gpu_executor.py:76] # GPU blocks: 61549, # CPU blocks: 4681
INFO 01-19 10:44:44 gpu_executor.py:80] Maximum concurrency for 32768 tokens per request: 30.05x
INFO 01-19 10:44:47 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_util

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:12<00:00,  2.83it/s]

INFO 01-19 10:44:59 model_runner.py:1535] Graph capturing finished in 12 secs, took 0.22 GiB
INFO 01-19 10:44:59 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 17.17 seconds


Делаем цепочку для обработки запроса.

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""
Вы являетесь помощником при выполнении заданий по поиску ответов на вопросы.
Используйте приведенные ниже фрагменты извлеченного контекста и только их, чтобы ответить на вопрос.
Если в приведенном контексте нет ответа на вопрос, сообщите об этом.
Ответ должен быть на русском языке.

<context>
Контекст: {context}
<\context>

Вопрос: {input}
Ответ:
""")

document_chain = create_stuff_documents_chain(llm_model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)


<>:24: SyntaxWarning: invalid escape sequence '\c'
<>:24: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipykernel_201890/674524840.py:24: SyntaxWarning: invalid escape sequence '\c'
  prompt = ChatPromptTemplate.from_template("""


Посмотрим на несколько примеров ответов

In [6]:
response = retrieval_chain.invoke({"input": "в чем заключается задача классификации?"})
print(response["answer"])

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.72s/it, est. speed input: 340.49 toks/s, output: 83.64 toks/s]

Задача классификации заключается в том, чтобы разделять объекты на различные классы на основе их признаков. В контексте линейной классификации это означает нахождение гиперплоскости, которая максимально разделяет объекты разных классов. Основная цель — минимизировать ошибку классификации и обеспечить, чтобы между классами была достаточно широкая зона разделения, называемая зазором (margin). В конкретном контексте, описанном в тексте, задача классификации включает в себя:

1. Нахождение вектора весов \( w \) и смещения \( w_0 \) таких, чтобы гиперплоскость \( \langle w, x_i \rangle - w_0 \) максимально разделяла классы, удовлетворяя условию \( y_i(\langle w, x_i \rangle - w_0) > 0 \) для всех \( i \).
2. Максимизация зазора между гиперплоскостью и ближайшими объектами из каждого класса для улучшения качества разделения.
3. Использование стратегий, таких как One vs One, для решения сложных случаев, когда невозможно разделить все классы одной гиперплоскостью.


In [7]:
response = retrieval_chain.invoke({"input": "в чем состоит идея бустинга?"})
print(response["answer"])

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.83s/it, est. speed input: 158.73 toks/s, output: 85.80 toks/s]

Идея бустинга заключается в построении последовательности слабых классификаторов, которые последовательно "усиливаются" путем корректировки весов объектов в обучающей выборке. Каждый новый классификатор в этой последовательности обучается на модифицированном наборе данных, на котором предыдущие классификаторы ошиблись. Таким образом, создается более мощный композитный классификатор, который, в результате комбинирования слабых классификаторов, способен более точно отделять классы в данных, особенно в тех случаях, когда классы нелинейно разделимы или имеют сложные границы. В данном контексте, точная информация о бустинге отсутствует, так как он не упоминается в приведенном фрагменте текста. Контекст касается линейных классификаторов и их проблем при решении задач классификации, однако идея бустинга, которая связана с построением ансамблей слабых классификаторов, в этом фрагменте не рассматривается. 

Для ответа на вопрос предлагается обратиться к другим контекстам или литературе по теме 

In [8]:
response = retrieval_chain.invoke({"input": "для чего применяется метод SVM?"})
print(response["answer"])

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.88s/it, est. speed input: 213.99 toks/s, output: 84.98 toks/s]

Метод SVM (Support Vector Machine) применяется для решения задач классификации и регрессии. Он находит гиперплоскость (или гиперповерхность в многомерном пространстве), которая максимально разделяет объекты разных классов. В задачах классификации SVM стремится создать границу, которая с наименьшей вероятностью будет содержать объекты из разных классов, тем самым минимизируя обобщающую ошибку. В задачах регрессии SVM может использоваться для построения границы, которая наилучшим образом аппроксимирует зависимость между входными и выходными данными.

Однако в данном контексте нет конкретной информации о том, как применяется метод SVM, поэтому давайте выясним это поэтапно на основе контекста:

1. **Тренировочная и валидационная выборки**: Метод SVM часто сочетается с методами разделения данных на тренировочную и валидационную выборки. Например, использование train-test split или кросс-валидации помогает определить оптимальные гиперпараметры, такие как размер ядра или уровень регуляризации

## промежуточные выводы
модель генерации текста работает хорошо, однако модель векторизации текста подтупливает и не находит нужных фрагментов.

In [22]:
from RAG.data_parser import preprocess_latex, parse_teormin_text

filename = "5_sem_ml/section/polidobro/teormin.tex"
qa_teormin = parse_teormin_text(filename)

qa_teormin[:2]

[['Постановка задач обучения с учителем (supervised learning).',
  ' Постановка задачи с <<учителем>> подразумевает наличие целевых меток для предсказания (targets). Определим следующее:  Training set , где    для регрессии   для бинарной классификации   для многоклассовой классификации   Модель , которая предсказывает какое-то значение для каждого объекта.  Функцию потерь , которую мы будем минимизировать.  Формальнее, пусть имеется семейство моделей . Допустим, что оно параметризовано (можно и без параметризации) вектором , тогда задача оптимизации: найти , что . Как пример, линейные регрессионные модели параметризуются векторами весов,  (для bias term). Примеры задач обучения с учителем:   Задача классификации~--- задача обучения с учителем. У нас есть набор классов, для некоторого множества объектов есть ответы (знаем к какому классу они принадлежат), для некоторого другого множества нужно предсказать класс. Пример: предсказание вернет клиент банка кредит или нет по историческим да

In [13]:
model_answers = []
for qa in qa_teormin[:10]:
    question, answer = qa
    response = retrieval_chain.invoke({"input": f"{question}"})
    model_answer = response["answer"]
    line = [question, answer, model_answer]
    model_answers.append(line)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.03s/it, est. speed input: 481.68 toks/s, output: 83.41 toks/s]


In [23]:
model_answers[:2]

[['Постановка задач обучения с учителем (supervised learning).',
  ' Постановка задачи с <<учителем>> подразумевает наличие целевых меток для предсказания (targets). Определим следующее:  Training set , где    для регрессии   для бинарной классификации   для многоклассовой классификации   Модель , которая предсказывает какое-то значение для каждого объекта.  Функцию потерь , которую мы будем минимизировать.  Формальнее, пусть имеется семейство моделей . Допустим, что оно параметризовано (можно и без параметризации) вектором , тогда задача оптимизации: найти , что . Как пример, линейные регрессионные модели параметризуются векторами весов,  (для bias term). Примеры задач обучения с учителем:   Задача классификации~--- задача обучения с учителем. У нас есть набор классов, для некоторого множества объектов есть ответы (знаем к какому классу они принадлежат), для некоторого другого множества нужно предсказать класс. Пример: предсказание вернет клиент банка кредит или нет по историческим да

Отобразим в виде таблицы. 

Видим, что модель на все отвечает хорошо. Однако местами нужный контекст не находится. Возможно это связано со способом построения эмбеддингов для текстов.

In [20]:
# This template helps to compare generated code samples in pretty table form
# feel free to present your work in other forms

from IPython.display import HTML, display

table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PROMPT</th>
    <th style="text-align: center; border:1px solid black">Answer</th>
    <th style="text-align: center; border:1px solid black">Model answer</th>
  </tr>
{}
</table>"""

row_template = """  <tr>
    <td style="width:15%; border:1px solid black; padding: 5px; white-space: pre-wrap;">{}</td>
    <td style="width:35%; border:1px solid black; padding: 5px; white-space: pre-wrap;">{}</td>
    <td style="width:40%; border:1px solid black; padding: 5px; white-space: pre-wrap;">{}</td>
  </tr>"""

rows = []

show_first = 300
for qas in model_answers:
    question, answer, model_answer = qas
    # replace placeholders in the format() arguments
    rows.append(
        row_template.format(
            question, answer[:show_first].replace("<s>", ""), model_answer[:show_first].replace("<s>", "")
        )
    )

display(HTML(table_template.format("\n".join(rows))))

PROMPT,Answer,Model answer
Постановка задач обучения с учителем (supervised learning).,"Постановка задачи с <<учителем>> подразумевает наличие целевых меток для предсказания (targets). Определим следующее: Training set , где для регрессии для бинарной классификации для многоклассовой классификации Модель , которая предсказывает какое-то значение для каждого объекта. Функцию","Обучение с учителем (supervised learning) включает в себя задачи, в которых имеется таргет, или целевая переменная, и цель состоит в том, чтобы научить модель предсказывать эту переменную на основе входных данных. В контексте текста, приведенном выше, задачи, которые обсуждаются, включают: 1. **Лин"
Задачи обучения без учителя. Назвать хотя бы две.,"Задача обучения без учителя: у нас нет таргета. Примеры таких задач: Задача кластеризации~--- задача обучения без учителя. Есть множество объектов нужно разбить их на группы так, чтобы “похожие” объекты оказались в одной, а непохожие в разных. Пример: есть разнородное множество объектов, для кото","Задачи обучения без учителя можно назвать: 1. Задача кластеризации: включает в себя разбиение множества объектов на группы на основе схожести между ними, без наличия заранее известных меток. Примером может служить кластеризация покупателей магазина одежды для рекомендаций товаров. 2. Уменьшение разм"
Что означает свойство i.i.d.?,"i.i.d = независимые и одинаково распределенные Объект~--- -мерный вектор, порожденный из некоторого распределения (то есть случайной величиной), тут подразумевается независимость объектов как независимость таких случайных векторов. Одинаково распределенные: порождены одинаковым процессом (с одина","свойство i.i.d. означает ""независимые и одинаково распределенные"". Контекст не содержит данной информации. В предоставленном контексте обсуждаются темы, связанные с линейной регрессией, градиентным бустингом, регуляризацией, но нет упоминания о свойстве i.i.d. Чтобы ответить на вопрос, нужно обрати"
Основная идея наивного Байесовского классификатора. В чём его наивность?,"Основная идея: мы хотим использовать теорему Байеса. Для этого нам надо чтобы признаки были независимы (а это обычно неправда, например, температура в Цельсиях и Фаренгейтах).","Основная идея наивного Байесовского классификатора заключается в предположении о независимости всех признаков. Это позволяет факторизовать совместную вероятность, как произведение условных вероятностей каждого признака: $$P(x_i|y_i = C_k) = \prod\limits_{n = 1}^p P(x_i^n|y_i = C_k)$$ Наивность кла"
Запишите формулы для модели линейной регрессии и для среднеквадратичной ошибки.,"Пусть у нас есть матрица объектов и столбец таргетов Тогда модель линейной регрессии записывается так: f_w(X) = Xw = Y Среднеквадратичная ошибка записывается так: , где ~--- мощность выборки. Оптимальная оценка для просто квадратичной нормы ищется так: . Рассмотрим ^2 = (X - Z)^T(X - Z) = X^","Для модели линейной регрессии формула выглядит следующим образом: \[ \hat{w} = (X^TX + \lambda I)^{-1}X^TY \] где: - \(\hat{w}\) — вектор весов, - \(X\) — матрица признаков, - \(Y\) — вектор целевых значений, - \(\lambda\) — гиперпараметр, регулирующий важность минимизации нормы вектора весов, - \(I"
Запишите формулу для одного шага градиентного спуска. Как модифицировать градиентный спуск для очень большой выборки?,"Градиент~--- вектор, направление которого совпадает с направлением наибольшего возрастания величины , значение которой меняется от одной точки пространства к другой (скалярного поля), а по величине (модулю) равный скорости роста этой величины в этом направлении. ~--- направление наибольшего убыва","Данная задача связана с объяснением градиентного спуска в контексте линейной регрессии с регуляризацией, но не содержит прямого упоминания о градиентном спуске в представленном контексте. Текст касается регуляризации, особенно в контексте нахождения аналитического решения методом наименьших квадрато"
"Что такое правдоподобие, метод максимального правдоподобия? Является ли правдопод